In [2]:
!pip install python_terrier

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 107.6/107.6 kB 2.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 13.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.6/69.6 kB 6.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 335.2/335.2 kB 15.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.8/48.8 kB 5.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 287.4/287.4 kB 15.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.1/45.1 kB 5.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.7/41.7 kB 5.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 25.3 MB/s eta 0:00:00
  Preparing metadata (setup.

In [3]:
import pyterrier as pt
if not pt.started():
  pt.init()


terrier-assemblies 5.8 jar-with-dependencies not found, downloading to /root/.pyterrier...
Done
terrier-python-helper 0.0.8 jar not found, downloading to /root/.pyterrier...
Done


PyTerrier 0.10.0 has loaded Terrier 5.8 (built by craigm on 2023-11-01 18:05) and terrier-helper 0.0.8



In [4]:
!pip install Arabic_StopWords

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 360.5/360.5 kB 5.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 126.4/126.4 kB 9.7 MB/s eta 0:00:00


In [5]:
import pandas as pd
pd.set_option('display.max_colwidth', 150)
import numpy as np
import re
from snowballstemmer import stemmer
from tqdm import tqdm
import arabicstopwords.arabicstopwords as stp

In [6]:
#removing Stop Words function
def remove_stopWords(sentence):
    terms=[]
    stopWords= set(stp.stopwords_list())
    for term in sentence.split() :
        if term not in stopWords :
           terms.append(term)
    return " ".join(terms)

#a function to normalize the tweets
def normalize(text):
    text = re.sub("[إأٱآا]", "ا", text)
    text = re.sub("ى", "ي", text)
    text = re.sub("ؤ", "ء", text)
    text = re.sub("ئ", "ء", text)
    text = re.sub("ة", "ه", text)
    return(text)

#define the stemming function
ar_stemmer = stemmer("arabic")
def stem(sentence):
    return " ".join([ar_stemmer.stemWord(i) for i in sentence.split()])

In [7]:
# %rm -rf IR-in-Arabic
# %rm -rf evetarIndex
!git clone https://github.com/telsayed/IR-in-Arabic.git
!unzip IR-in-Arabic/Summer2021/data/EveTAR/evetarIndex.zip -d evetarIndex
!ls evetarIndex

Cloning into 'IR-in-Arabic'...
remote: Enumerating objects: 710, done.
remote: Counting objects: 100% (49/49), done.
remote: Compressing objects: 100% (49/49), done.
remote: Total 710 (delta 12), reused 0 (delta 0), pack-reused 661
Receiving objects: 100% (710/710), 43.94 MiB | 21.37 MiB/s, done.
Resolving deltas: 100% (189/189), done.
Archive:  IR-in-Arabic/Summer2021/data/EveTAR/evetarIndex.zip
  inflating: evetarIndex/data.direct.bf  
  inflating: evetarIndex/data.document.fsarrayfile  
  inflating: evetarIndex/data.inverted.bf  
  inflating: evetarIndex/data.lexicon.fsomapfile  
  inflating: evetarIndex/data.lexicon.fsomaphash  
  inflating: evetarIndex/data.lexicon.fsomapid  
  inflating: evetarIndex/data.meta.idx  
  inflating: evetarIndex/data.meta.zdata  
  inflating: evetarIndex/data.properties  
data.direct.bf		   data.lexicon.fsomapfile  data.meta.idx
data.document.fsarrayfile  data.lexicon.fsomaphash  data.meta.zdata
data.inverted.bf	   data.lexicon.fsomapid    data.propert

In [8]:
index_ref = pt.IndexRef.of("./evetarIndex/data.properties")
index = pt.IndexFactory.of(index_ref)

### **Vector space model retrieval**

In [9]:
tfidfreter=pt.BatchRetrieve(index,controls={"wmodel":"TF_IDF"},num_of_result=10)

In [10]:
def preprocess(sentence):
  sentence=remove_stopWords(sentence)
  sentence=normalize(sentence)
  sentence=stem(sentence)

  return sentence

### test term

In [11]:
query="الشرق الاوسط"
query = preprocess(query)
result=tfidfreter.search(query)
result

BR(TF_IDF):   0%|          | 0/1 [00:00<?, ?q/s]

,qid,docid,docno,rank,score,query
0,1,41057,560060033028071425,0,9.296541,شرق اوسط
1,1,16661,553468445745221632,1,8.883980,شرق اوسط
2,1,18726,553591317801234433,2,8.883980,شرق اوسط
3,1,41437,560105503406030849,3,8.661938,شرق اوسط
4,1,41439,560105510964170752,4,8.661938,شرق اوسط
...,...,...,...,...,...,...
459,1,43436,560487293069590528,459,2.810185,شرق اوسط
460,1,46575,561074472761827328,460,2.810185,شرق اوسط
461,1,45761,560930191284129792,461,2.731100,شرق اوسط
462,1,45762,560930192181325824,462,2.731100,شرق اوسط


### read data

In [12]:
dataset_links=["https://raw.githubusercontent.com/telsayed/IR-in-Arabic/master/Summer2021/data/EveTAR/tweets/evetar-q-01.txt",
               "https://raw.githubusercontent.com/telsayed/IR-in-Arabic/master/Summer2021/data/EveTAR/tweets/evetar-q-02.txt",
               "https://raw.githubusercontent.com/telsayed/IR-in-Arabic/master/Summer2021/data/EveTAR/tweets/evetar-q-03.txt",
               "https://raw.githubusercontent.com/telsayed/IR-in-Arabic/master/Summer2021/data/EveTAR/tweets/evetar-q-04.txt",
               "https://raw.githubusercontent.com/telsayed/IR-in-Arabic/master/Summer2021/data/EveTAR/tweets/evetar-q-05.txt",
               "https://raw.githubusercontent.com/telsayed/IR-in-Arabic/master/Summer2021/data/EveTAR/tweets/evetar-q-06.txt",
               "https://raw.githubusercontent.com/telsayed/IR-in-Arabic/master/Summer2021/data/EveTAR/tweets/evetar-q-07.txt",
               "https://raw.githubusercontent.com/telsayed/IR-in-Arabic/master/Summer2021/data/EveTAR/tweets/evetar-q-08.txt",
               "https://raw.githubusercontent.com/telsayed/IR-in-Arabic/master/Summer2021/data/EveTAR/tweets/evetar-q-09.txt",
               "https://raw.githubusercontent.com/telsayed/IR-in-Arabic/master/Summer2021/data/EveTAR/tweets/evetar-q-10.txt"]
full_data=pd.DataFrame()
for i in tqdm(range(len(dataset_links))):
    tweets=pd.read_csv(dataset_links[i], sep='\t')
    full_data=pd.concat([full_data,tweets],ignore_index=True)
full_data.reset_index(inplace=True,drop=True)
#the docno will be our tweetID
full_data["docno"]=full_data["tweetID"].astype(str)
#select tweet text for the tweets retrieved only
full_data[full_data["docno"].isin(result["docno"].tolist())]

100%|██████████| 10/10 [00:02<00:00,  3.35it/s]


,tweetID,tweetText,docno
4,549720880717508608,البحرين: ضبط مطلوبين متورطين في التفجير بالعكر الشرقي بقية الموضوع اضغط هنا http://t.co/t4A5bNrqyh,549720880717508608
230,550216877788921856,امتنعت المملكة المتحدة، ورواندا، ونيجيريا، ليتوانيا، وكوريا الجنوبية عن التصويت لمشروع قرار دولة فلسطين #تسريب_الشرق_لمكتب_السيسي,550216877788921856
719,550348944350277632,#الشرق عباس يوقع على الانضمام الى المحكمة الجنائية الدولية والى 20 منظمة واتفاقية دولية http://t.co/ffoDmUHR45,550348944350277632
733,550352202918023168,عباس يوقع على الانضمام الى المحكمة الجنائية الدولية والى 20 منظمة واتفاقية دولية http://t.co/baSdobCvy2 #الشرق_الاوسط,550352202918023168
1203,550521908593041409,في بداية سيئة للسلام في الشرق الأوسط، رئيس الوزراء الإسرائيلي بنيامين نتنياهو يفوز بترشيح حزب الليكود للانتخابات القادمة.,550521908593041409
...,...,...,...
48476,561403881809125376,الشرق الأسط: #سلمان_الدوسري: إعادة هيكلة #الدولة #السعودية http://t.co/sDPqaCl2Av @SalmanAldosary,561403881809125376
48949,561461417787023362,#جند_الله تتبنى #تفجير #مسجدٍ في #باكستان خلَّف 44 قتيلاً http://t.co/0aQQDXjmHq #إسلام_أباد #السعودية #الشرق,561461417787023362
49076,561508379596570624,⚪ مقتل امير قبيلة عنزة في العراق بتفجير انتحاري #العراق #الشرق_الأوسط http://t.co/jJhAWRz3dL,561508379596570624
49447,561622227108380672,#بوابة_الشرق | إصابة 3 أشخاص في تفجير سيارة وسط إسرائيل http://t.co/7kykMJTZSv,561622227108380672


In [13]:
query="مصر"
query=preprocess(query)
result=tfidfreter.search(query)
result

BR(TF_IDF):   0%|          | 0/1 [00:00<?, ?q/s]

,qid,docid,docno,rank,score,query
0,1,29758,556299087591391232,0,4.600152,مصر
1,1,49112,561524708839927808,1,4.404388,مصر
2,1,49113,561524708932210689,2,4.404388,مصر
3,1,7999,551991253706211328,3,4.395339,مصر
4,1,4983,551568925901615104,4,4.318752,مصر
...,...,...,...,...,...,...
995,1,26190,554713632739557376,995,2.506976,مصر
996,1,27052,554907852859772928,996,2.506976,مصر
997,1,27054,554908236022038529,997,2.506976,مصر
998,1,27056,554908248176726018,998,2.506976,مصر


In [14]:
full_data[full_data["docno"].isin(result["docno"].tolist())]

,tweetID,tweetText,docno
91,549957439207272449,#مصر تدين حادث التفجير الإرهابي في طبرق: لإفشال الجهود الدولية - صحيفة المشهد http://t.co/sdJFhmniXm,549957439207272449
92,549959309556461568,مصر تدين حادث التفجير الإرهابى فى مدينة طبرق الليبية http://t.co/NVDMnfXB1u,549959309556461568
95,549971439634628609,#مصر تدين حادث التفجير الإرهابي في طبرق وتؤكد دعمها للمؤسسات الشرعية في ليبيا - بوابة الشروق http://t.co/ydGhVf2Ief,549971439634628609
98,549973848041345025,#عاجل: الخارجية تدين حادث التفجير الإرهابي في طبرق http://t.co/d2l4Wt2Gni #مصر,549973848041345025
125,550041919993020417,بوابة الشروق| #مصر تدين حادث التفجير الإرهابي في طبرق وتؤكد دعمها للمؤسسات الشرعية في ليبيا http://t.co/oUpyk7AP0L,550041919993020417
...,...,...,...
49775,561880588249223168,#كلنا_الجيش_المصرى تفجير قنبلة في شارع المشير,561880588249223168
49784,561882878376935424,تفجير ٣منازل مره واحده ف جانب المصري,561882878376935424
49836,561902846082564097,أسرة معتقل مصري : الشرطة قتلته و إتهمته بتفجير مجهول !! WTF http://t.co/ZQuUIL8W1n,561902846082564097
49846,561905183224233984,أسرة معتقل مصري: الشرطة قتلته واتهمته بتفجير مجهول http://t.co/pxN8WNYH4y عبر @sharethis,561905183224233984


In [15]:
topics=pd.read_csv("https://raw.githubusercontent.com/telsayed/IR-in-Arabic/master/Summer2021/data/EveTAR/topics.txt", sep='\t',names=['data'])
queries=[]
qid=[]
#we will get the queries and their ids from the topics file
for i in range(len(topics)):
    splitted=topics["data"][i:i+1][i].split(" ")
    if splitted[0]=="<title>":
       queries.append(' '.join(splitted[1:]))
    if splitted[0]=="<num>":
       qid.append(splitted[2])

queriesDF=pd.DataFrame()
queriesDF["qid"]=qid
queriesDF["raw_query"]=queries
#remove the stopwords from queries, do normalization, and apply stemming
queriesDF["query"]=queriesDF["raw_query"].apply(preprocess)

queriesDF

,qid,raw_query,query
0,E01,تفجير انتحاري في اب,تفجير انتحار اب
1,E02,ليتوانيا تستخدم اليورو بدل الليتاس,ليتواني استخدم يورو بدل ليتاس
2,E03,فلسطين تطلب الانضمام للمحكمة الجنائية الدولية,لسط تطلب انضمام محكمه جناءيه دوليه
3,E04,وفاة أبو أنس الليبي في نيويورك,وفا انس ليب نيويور
4,E05,اختراق كوريا الشمالية حسابات سوني,اختراق كوري شماليه حسابا سون
5,E06,بناء أول كنيسة في إسطنبول منذ قرن,ناء كنيس اسطنبول قرن
6,E07,باباندريو يؤسس حزب جديد,باباندر يءسس حزب جديد
7,E08,بوكو حرام تخطف شباب في نيجريا,وكو حرام تخطف شباب نيجري
8,E09,سيطرة بوكو حرام على قاعدة عسكرية في نيجيريا,سيطر وكو حرام قاعد عسكر نيجيري
9,E10,مقتل مسلحين في غارات في باكستان,مقتل مسلح غارا باكست


In [17]:
tfidfreter=tfidfreter.transform(queriesDF)
tfidfreter

BR(TF_IDF):   0%|          | 0/50 [00:00<?, ?q/s]

,qid,docid,docno,rank,score,raw_query,query
0,E01,1602,550652866793578496,0,10.389893,تفجير انتحاري في اب,تفجير انتحار اب
1,E01,420,550265054273097729,1,10.285878,تفجير انتحاري في اب,تفجير انتحار اب
2,E01,329,550239745607880704,2,9.907399,تفجير انتحاري في اب,تفجير انتحار اب
3,E01,453,550274033946988544,3,9.907399,تفجير انتحاري في اب,تفجير انتحار اب
4,E01,1376,550582854749790208,4,9.907399,تفجير انتحاري في اب,تفجير انتحار اب
...,...,...,...,...,...,...,...
48126,E65,830,550377113019097088,995,3.474196,إعادة تشكيل مجلس الوزراء السعودي,اعاد تشكيل مجلس وزراء سعود
48127,E65,12652,552853899498250241,996,3.474196,إعادة تشكيل مجلس الوزراء السعودي,اعاد تشكيل مجلس وزراء سعود
48128,E65,13958,553090384524742656,997,3.474196,إعادة تشكيل مجلس الوزراء السعودي,اعاد تشكيل مجلس وزراء سعود
48129,E65,23083,554205833538854913,998,3.474196,إعادة تشكيل مجلس الوزراء السعودي,اعاد تشكيل مجلس وزراء سعود


In [18]:
#specify BM25 as wmodel
bm25_retr = pt.BatchRetrieve(index, controls = {"wmodel": "BM25"},num_results=10)
bm25_res=bm25_retr.transform(queriesDF)
bm25_res

,qid,docid,docno,rank,score,raw_query,query
0,E01,1602,550652866793578496,0,18.524609,تفجير انتحاري في اب,تفجير انتحار اب
1,E01,420,550265054273097729,1,18.339155,تفجير انتحاري في اب,تفجير انتحار اب
2,E01,329,550239745607880704,2,17.664349,تفجير انتحاري في اب,تفجير انتحار اب
3,E01,453,550274033946988544,3,17.664349,تفجير انتحاري في اب,تفجير انتحار اب
4,E01,1376,550582854749790208,4,17.664349,تفجير انتحاري في اب,تفجير انتحار اب
...,...,...,...,...,...,...,...
495,E65,47226,561151529491906560,5,31.127143,إعادة تشكيل مجلس الوزراء السعودي,اعاد تشكيل مجلس وزراء سعود
496,E65,45134,560878119356100608,6,30.646378,إعادة تشكيل مجلس الوزراء السعودي,اعاد تشكيل مجلس وزراء سعود
497,E65,45243,560882023556866048,7,30.218171,إعادة تشكيل مجلس الوزراء السعودي,اعاد تشكيل مجلس وزراء سعود
498,E65,45145,560878231834722304,8,29.905221,إعادة تشكيل مجلس الوزراء السعودي,اعاد تشكيل مجلس وزراء سعود


### **Evaluating our results**

In [19]:
qrels=pd.read_csv("https://raw.githubusercontent.com/telsayed/IR-in-Arabic/master/Summer2021/data/EveTAR/qrels.txt", sep='\t',names=['qid','Q0','docno','label'])
qrels['docno']=qrels['docno'].astype(str)
# qrels are in TREC format
qrels = qrels[qrels["docno"].isin(full_data["docno"].tolist())] # to choose qrels for the chosen 50k documents
qrels

,qid,Q0,docno,label
0,E01,Q0,549711593487888387,0
1,E01,Q0,549719610459967488,0
2,E01,Q0,549720880717508608,0
4,E01,Q0,549780004230283264,0
7,E01,Q0,549880138033135617,0
...,...,...,...,...
61940,E65,Q0,561984084948176896,0
61941,E65,Q0,561984303219744768,0
61942,E65,Q0,561984392516472833,0
61944,E65,Q0,561984778438590464,0


 example of a dataset with graded relevance judgement.

In [20]:
pt.get_dataset("trec-robust-2004").get_qrels()

qrels.robust2004.txt:   0%|          | 0.00/6.24M [00:00<?, ?iB/s]

,qid,docno,label
0,301,FBIS3-10082,1
1,301,FBIS3-10169,0
2,301,FBIS3-10243,1
3,301,FBIS3-10319,0
4,301,FBIS3-10397,1
...,...,...,...
311405,700,LA122690-0089,0
311406,700,LA122789-0005,0
311407,700,LA122889-0096,0
311408,700,LA122990-0108,0


In [21]:
pt.get_dataset("trec-robust-2004").get_qrels()['label'].unique()

array([1, 0, 2])

 we are evaluating TF_IDF retrieval model

In [22]:
eval = pt.Utils.evaluate(bm25_res,qrels[['qid','docno','label']],metrics=["map","recall","P"],perquery=True)
eval

<ipython-input-22-ff733742c09e>:1: DeprecationWarning: Call to deprecated function (or staticmethod) evaluate. (Use pt.Evaluate instead) -- Deprecated since version 0.9.
  eval = pt.Utils.evaluate(bm25_res,qrels[['qid','docno','label']],metrics=["map","recall","P"],perquery=True)


defaultdict(dict,
            {'E01': {'map': 0.015127175368139225,
              'P@5': 1.0,
              'P@10': 0.9,
              'P@15': 0.6,
              'P@20': 0.45,
              'P@30': 0.3,
              'P@100': 0.09,
              'P@200': 0.045,
              'P@500': 0.018,
              'P@1000': 0.009,
              'R@5': 0.008605851979345954,
              'R@10': 0.01549053356282272,
              'R@15': 0.01549053356282272,
              'R@20': 0.01549053356282272,
              'R@30': 0.01549053356282272,
              'R@100': 0.01549053356282272,
              'R@200': 0.01549053356282272,
              'R@500': 0.01549053356282272,
              'R@1000': 0.01549053356282272},
             'E02': {'map': 0.058823529411764705,
              'P@5': 1.0,
              'P@10': 1.0,
              'P@15': 0.6666666666666666,
              'P@20': 0.5,
              'P@30': 0.3333333333333333,
              'P@100': 0.1,
              'P@200': 0.05,
             